# 卷积网络的参数定义

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [8]:
# # 学习dropout的使用方法 https://pytorch.org/docs/stable/generated/torch.nn.Dropout.html 
# # dropout层，输出与输入维度相同，若干elements置0，其他elements乘以一个缩放因子（1/（1-p））
# # dropout的作用，在训练的准确率比较低时，测试的准确率比较高（测试时不drop，若干弱分类器的集合）
# # 一般在全连接层前加一个dropout，卷积层不太需要，卷积ReLU相当于已经有了部分置0
# m = nn.Dropout(p=0.5)
# input = torch.tensor([1,2,3,4,5,6], dtype=torch.float32)
# output = m(input)
# print(output)

## 读取数据

In [9]:
from torchvision.transforms.transforms import ToTensor
# 定义超参数
input_size = 28    # 图像尺寸
num_classes = 10    # 标签的种类数
num_epochs = 10    # 训练的总轮数
batch_size = 64    # 一个批的大小64张照片

# pytorch自带了MNIST的数据集
# 训练集
train_dataset = datasets.MNIST(root = './data',
                train = True,
                transform = transforms.ToTensor(),
                download = True)

# 测试集
test_dataset = datasets.MNIST(root = './data',
                train = False,
                transform = transforms.ToTensor(),
                download = True)

# 构建batch数据
train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                      batch_size = batch_size,
                      shuffle = True)

test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                      batch_size = batch_size,
                      shuffle = False)

## 构建卷积网络模块

In [10]:
class CNN(nn.Module):
  def __init__(self):
    super().__init__()

    # 输入大小(1,28,28)
    self.conv1 = nn.Sequential(
        nn.Conv2d(
            in_channels=1,      # 输入灰度图
            out_channels=16,     # 输出特征图通道数
            kernel_size=5,      # 卷积核大小
            stride=1,         # 步长
            padding=2         # 根据卷积核大小，padding2时输出与原图一样大，填充0
        ),                # 输出特征图大小为(16,28,28)
        nn.ReLU(),            # 激活层
        nn.MaxPool2d(kernel_size=2)    # 选择2*2区域池化，输出特征图大小(16,14,14)
    )
    
    # 输入大小(16,14,14)
    self.conv2 = nn.Sequential(
        nn.Conv2d(16, 32, 5, 1, 2),  # 输出特征图大小为(32,14.14)
        nn.ReLU(),             # 激活层
        nn.MaxPool2d(2)           # 输出特征图大小(32,7,7)
    )

    # 期待输入大小32*7*7
    self.dropout = nn.Dropout(p=0.5)     # dropout层，训练时随机删除一些神经元，测试时不删除
    self.fc = nn.Linear(32*7*7, 10)     # 全连接层，输入卷积特征图拉长结果，输出10个值

  def forward(self, x):
    # print("输入前：",x.size())         # 输入前： torch.Size([64, 1, 28, 28])
    x = self.conv1(x)
    # print("经过conv1：",x.size())        # 经过conv1： torch.Size([64, 16, 14, 14])
    x = self.conv2(x)
    # print("经过conv2：",x.size())        # 经过conv2： torch.Size([64, 32, 7, 7])
    x = x.view(x.size(0), -1)         # 输入全连接层前需要改变形状
    # print("输入全连接层的：",x.size())      # 输入全连接层的： torch.Size([64, 1568])
    x = self.dropout(x)
    output = self.fc(x)
    return output

# 参考链接：
# numpy与pytorch中的shape与size： https://blog.csdn.net/sinat_28442665/article/details/113743138 
# pytorch中 view与reshape的区别：https://blog.csdn.net/Flag_ing/article/details/109129752
# numpy中 resize是在原数据上操作，无返回值；reshape返回一个新的数据

## 统计准确率函数

In [11]:
def accuracy(prediction, labels):
  pred = torch.max(prediction.data, 1)[1]    # [0]是values，[1]是index
  rights = pred.eq(labels.data.view_as(pred)).sum()
  return rights, len(labels)

## 训练网络模型

In [ ]:
# 实例化
net = CNN()
# 损失函数
criterion = nn.CrossEntropyLoss()
# 优化器
optimizer = optim.Adam(net.parameters(), lr=0.001)

# 开始循环训练
for epoch in range(num_epochs):
  # 定义一个列表，保存当前结果
  train_rights = []

  for batch_idx, (data, target) in enumerate(train_loader):
    net.train()
    output = net(data)
    loss = criterion(output, target)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    right = accuracy(output, target)
    train_rights.append(right)

    if batch_idx % 100 == 0:
      net.eval()
      test_rights = []
      for (data, target) in test_loader:
        output = net(data)
        rught = accuracy(output, target)
        test_rights.append(right)

      # 准确率计算
      train_r = (sum(bar[0] for bar in train_rights), sum(bar[1] for bar in train_rights))
      test_r = (sum(bar[0] for bar in test_rights), sum(bar[1] for bar in test_rights))


      # 打印信息
      print("epoch{} : [{}/{} ({:.0f}/%)]\t损失: {:.4f}\t训练准确率: {:.2f}/%\t测试准确率: {:.2f}/%".format(
          epoch, batch_idx*batch_size, len(train_dataset),
          100.0 * batch_idx*batch_size / len(train_dataset),
          loss.item(),
          100 * train_r[0].item() / train_r[1],
          100 * test_r[0].item() / test_r[1],
      )) 


epoch0 : [0/60000 (0/%)]	损失: 2.2962	训练准确率: 9.38/%	测试准确率: 9.38/%
epoch0 : [6400/60000 (11/%)]	损失: 0.7426	训练准确率: 70.64/%	测试准确率: 81.25/%
epoch0 : [12800/60000 (21/%)]	损失: 0.1628	训练准确率: 80.84/%	测试准确率: 95.31/%
epoch0 : [19200/60000 (32/%)]	损失: 0.2094	训练准确率: 85.01/%	测试准确率: 93.75/%
epoch0 : [25600/60000 (43/%)]	损失: 0.0638	训练准确率: 87.46/%	测试准确率: 96.88/%
epoch0 : [32000/60000 (53/%)]	损失: 0.1682	训练准确率: 89.16/%	测试准确率: 93.75/%
epoch0 : [38400/60000 (64/%)]	损失: 0.0405	训练准确率: 90.29/%	测试准确率: 100.00/%
